In [1]:
import nbformat as nbf
from nbformat import read, NO_CONVERT
import nbconvert as nbc
from nbconvert.preprocessors import ExecutePreprocessor
import sys, os

# Disable file validation (useful for some IDEs)
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

# Define available cosmological models
cosmoin = ["LCDM", "fR4", "fR5", "fR6", "LCDM_0.15"]

# Select cosmology
c_run = 0

# Define base path of the repository
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Folder containing input maps for selected cosmology
folderin = os.path.join(base_path, "data", cosmoin[c_run])

# Path to the notebook to execute (assumed to be in same dir as this script)
notebook_path = os.path.join(os.getcwd(), "Tunnel_void_finder.ipynb")

# Number of lightcone maps
n = 256  
subfolders = [str(i).zfill(2) for i in range(n)]  # Generate a list of subfolders from "00" to "n"

# Loop over all subfolders and run the notebook
for l_c in subfolders:
    folder_path = os.path.join(folderin, l_c)
    print(f"Executing map n: {l_c} in folder: {folder_path}")

    if not os.path.isdir(folder_path):
        print(f"Folder not found: {folder_path}, skipping.")
        continue

    # Load the notebook
    with open(notebook_path, "r", encoding="utf-8") as f:
        nb = nbf.read(f, as_version=4)

    # Modify second cell: replace the line starting with `l_c =`
    try:
        second_cell = nb['cells'][1]
        second_cell_lines = second_cell['source'].split('\n')
        new_lines = [
            f'l_c = "{l_c}"' if line.strip().startswith("l_c =") else line
            for line in second_cell_lines
        ]
        nb['cells'][1]['source'] = '\n'.join(new_lines)
    except IndexError:
        print("Notebook does not have a second cell.")
        continue

    # Execute the notebook
    try:
        ep = ExecutePreprocessor(timeout=900, kernel_name="python3")
        ep.preprocess(nb, {'metadata': {'path': '.'}})
    except Exception as e:
        print(f"Error executing notebook for map {l_c}: {e}")

    # Return to root path before next iteration
    os.chdir(base_path)


Executing map n: 00 in folder: /home/leonardo/PyTwinPeaks/data/LCDM/00
